In [1]:
from sklearn.datasets import load_boston
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

import pandas as pd
import numpy as np

import optuna
from objective import Objective

#### 再現のための乱数値指定

In [2]:
SEED = 334

#### 学習データの読み込み

In [3]:
boston = load_boston()
df_X = pd.DataFrame(boston.data, columns = boston.feature_names, dtype = float)
df_y = pd.Series(boston.target, name = 'PRICE', dtype = float)

#### データの分割

In [4]:
X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size = 0.2, random_state = SEED)

#### インスタンス化

In [5]:
objective = Objective(RandomForestRegressor, X_train, y_train, random_state = SEED, cv = 5)

#### 最適化

In [6]:
study = optuna.create_study(direction = 'maximize')
study.optimize(objective, n_trials=10)

[I 2020-07-24 21:33:43,270] Finished trial#0 with value: -0.829707047842675 with parameters: {'min_samples_split': 16, 'max_depth': 471, 'n_estimators': 125}. Best is trial#0 with value: -0.829707047842675.
[I 2020-07-24 21:33:44,682] Finished trial#1 with value: -0.838345976960376 with parameters: {'min_samples_split': 11, 'max_depth': 14, 'n_estimators': 190}. Best is trial#1 with value: -0.838345976960376.
[I 2020-07-24 21:33:45,922] Finished trial#2 with value: -0.8385448018756453 with parameters: {'min_samples_split': 10, 'max_depth': 34, 'n_estimators': 155}. Best is trial#2 with value: -0.8385448018756453.
[I 2020-07-24 21:33:47,962] Finished trial#3 with value: -0.8409401817318157 with parameters: {'min_samples_split': 7, 'max_depth': 424, 'n_estimators': 268}. Best is trial#3 with value: -0.8409401817318157.
[I 2020-07-24 21:33:50,931] Finished trial#4 with value: -0.841994461367116 with parameters: {'min_samples_split': 7, 'max_depth': 408, 'n_estimators': 412}. Best is trial

#### 最適モデルを得る

In [7]:
best_clf = objective.clf(**study.best_params, **objective.fixed_params)

#### スコアで性能評価

In [8]:
best_clf.fit(X_train, y_train)
y_pred_on_test = best_clf.predict(X_test)
print(r2_score(y_test, y_pred_on_test))

0.8654697554552495
